In [105]:
import yfinance as yf
import scipy.stats as si
import pandas as pd
import numpy as np
import os

In [106]:
TSLA = yf.download("TSLA", start="2021-05-19", end="2022-05-19")

[*********************100%***********************]  1 of 1 completed


In [107]:
def newton_vol_call(S, K, T, C, r):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #C: Call value
    #r: risk free rate
    #sigma: volatility of underlying asset
   
    MAX_ITERATIONS = 100
    tolerance = 0.000001
    
    sigma = 1.25
    
    for i in range(0, MAX_ITERATIONS):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        price = S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
        vega = S * np.sqrt(T) * si.norm.pdf(d1, 0.0, 1.0)

        diff = C - price

        if (abs(diff) < tolerance):
            return sigma
        else: 
            sigma = sigma + diff/vega
        
        # print(i,sigma,diff)
        
    return sigma

In [108]:
impvol = newton_vol_call(714.15, 1000.00, 1/252, 0.01, 0.02)
print('The implied volatility is', round(impvol*100,2) , '% for the one-day call option with strike $ 1000.00' ) 

The implied volatility is 160.83 % for the one-day call option with strike $ 1000.00


In [109]:
log_return = np.log(TSLA['Adj Close'] / TSLA['Adj Close'].shift(1))
vol_h = np.sqrt(252) * log_return.std()
print('The annualised volatility is', round(vol_h*100,2), '%')

The annualised volatility is 57.51 %


In [110]:
def euro_option_bs(S, K, T, r, vol, payoff):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: risk free rate
    #vol: volatility of underlying asset
    #payoff: call or put
    
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    if payoff == "call":
        option_value = S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        option_value = - S * si.norm.cdf(-d1, 0.0, 1.0) + K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return option_value

In [111]:
T = euro_option_bs(714.15, 1000, 1/252, 0.02, 0.5751, 'call') #Call option price using the annualised volatility
print('The option price is $', T)

The option price is $ 2.4693009830212118e-20


In [112]:
T = euro_option_bs(714.15, 1000, 1/252, 0.02, 1.6083, 'call') #Call option price using the implied volatility
print('The option price is $', T)

The option price is $ 0.010001498707888745


In [113]:
S0 = 25.0               # spot stock price
K = 24.0                # strike
T = 6/12                # maturity 
r = 0.05                 # risk free rate 
sig = 0.20               # diffusion coefficient or volatility
N = 2                   # number of periods or number of time steps  
payoff = "call"          # payoff 

In [114]:
dT = float(T) / N                             # Delta t
u = np.exp(sig * np.sqrt(dT))                 # up factor
d = 1.0 / u                                   # down factor 

In [115]:
S = np.zeros((N + 1, N + 1))
S[0, 0] = S0
z = 1
for t in range(1, N + 1):
    for i in range(z):
        S[i, t] = S[i, t-1] * u
        S[i+1, t] = S[i, t-1] * d
    z += 1

In [116]:
S

array([[25.        , 27.62927295, 30.53506895],
       [ 0.        , 22.62093545, 25.        ],
       [ 0.        ,  0.        , 20.46826883]])

In [117]:
a = np.exp(r * dT)    # risk free compound return
p = (a - d)/ (u - d)  # risk neutral up probability
q = 1.0 - p           # risk neutral down probability
p

0.537808371956414

In [118]:
S_T = S[:,-1]
V = np.zeros((N + 1, N + 1))
if payoff =="call":
    V[:,-1] = np.maximum(S_T-K, 0.0)
elif payoff =="put":
    V[:,-1] = np.maximum(K-S_T, 0.0)
V

array([[0.        , 0.        , 6.53506895],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        ]])

In [119]:
# for European Option
for j in range(N-1, -1, -1):
    for i in range(j+1):
        V[i,j] = np.exp(-r*dT) * (p * V[i,j + 1] + q * V[i + 1,j + 1])
V

array([[2.32838692, 3.92740574, 6.53506895],
       [0.        , 0.53112761, 1.        ],
       [0.        , 0.        , 0.        ]])

In [120]:
print('European ' + payoff, str( V[0,0]))

European call 2.3283869193612463


In [121]:
def euro_option_bsm(S, K, T, r, q, vol, payoff):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: risk free rate
    #q: continuous dividend yield
    #vol: volatility of underlying asset
    #payoff: call or put
    
    d1 = (np.log(S / K) + (r - q + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    if payoff == "call":
        option_value = S * np.exp(-q * T) * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        option_value =  - S * np.exp(-q * T) * si.norm.cdf(-d1, 0.0, 1.0) + K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return option_value

In [122]:
euro_option_bsm(25, 24, 1/2, 0.05, 0, 0.20, 'call')

2.3055584059794327